In [48]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import s3fs
import json
from src import model_utils
from types import SimpleNamespace
import torch.nn.functional as F
with open("../UTAE_PAPs/conf.json") as f:
    config_dict = json.load(f)

with open("NORM_S2_patch.json") as f:
    norm_vals = json.load(f)
 
# Convertir le dict en objet (similaire à un namespace)
config = SimpleNamespace(**config_dict)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class S3SinglePatchDataset(Dataset):
    def __init__(self, s3_prefix, tile_ids, transform=None):
        self.s3_prefix = s3_prefix.rstrip("/")
        self.tile_ids = tile_ids
        self.transform = transform
        os.environ["AWS_ACCESS_KEY_ID"] = '1CH53NXGI0M8EML05AQY'
        os.environ["AWS_SECRET_ACCESS_KEY"] = 'YCz+C1hM3kokhoJTmISBFV84VHqpgmw4dUZ6Y5pk'
        os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiIxQ0g1M05YR0kwTThFTUwwNUFRWSIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzU0Mzc5NDA3LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6ImFudG9pbmUubGVzYXV2YWdlQGFncmljdWx0dXJlLmdvdXYuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzU0OTg0MzUzLCJmYW1pbHlfbmFtZSI6IkxFU0FVVkFHRSIsImdpdmVuX25hbWUiOiJBbnRvaW5lIiwiZ3JvdXBzIjpbIlVTRVJfT05ZWElBIl0sImlhdCI6MTc1NDM3OTU1MiwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDpmNTBmMDZkNC1kZWU1LWE4N2QtY2FhNC01MTg5NTkyZDVjZWQiLCJsb2NhbGUiOiJmciIsIm5hbWUiOiJBbnRvaW5lIExFU0FVVkFHRSIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJhbnRvaW5lbGVzYXV2YWdlIiwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLXNzcGNsb3VkIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXSwic2NvcGUiOiJvcGVuaWQgcHJvZmlsZSBncm91cHMgZW1haWwiLCJzaWQiOiJiODNkZWZiNC1kOWQwLTRhNWYtYjM3Ny1jMjI5NGZiYmMwZGEiLCJzdWIiOiJlOWY4NjIyYy01MzQ2LTRjOGUtOWZmYi1jY2MwMzZjY2ZjZjciLCJ0eXAiOiJCZWFyZXIifQ.v9xea_8hrBrC-X5PDukdL3hvn96XkN1cbmu9fHIlHd9_Hhe3OD68tjcCpfXdK22AvoKrrRCFseRjfyCwKzm--w'
        os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
        self.fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

    def __len__(self):
        return len(self.tile_ids)

    def __getitem__(self, idx):
        tile_id = self.tile_ids[idx]
        path = f"{self.s3_prefix}/{tile_id}.npy"
        with self.fs.open(path, 'rb') as f:
            arr = np.load(f)
            # Convertir en tensor PyTorch
        tensor = torch.from_numpy(arr).float()  # (T, C, H, W)

    # Appliquer padding/cropping
        #tensor = pad_or_crop(tensor, target_h=128, target_w=128) #AL commented out
        if self.transform:
            tensor = self.transform(tensor)
        return {"tile_id": tile_id, "data": tensor}  # (T, C, H, W)


def load_model_and_predict(tile_id, fold_num, return_raw_output=True):
    """Charge le modèle et fait une prédiction pour récupérer les instances"""

    # Réutiliser votre pipeline existant mais retourner les outputs bruts
    s3_prefix = "s3://antoinelesauvage/zenodo/PASTIS_unz/PASTIS/DATA_S2"
    dataset = S3SinglePatchDataset(s3_prefix, [f"S2_{tile_id}"])
    dataloader = DataLoader(dataset, batch_size=1)
    
    model = load_model_fold(fold_num)
    
    for batch in dataloader:
        data = batch["data"][0].unsqueeze(0).cuda()
        
        # Normalisation
        mean = torch.tensor(norm_vals[f"Fold_{fold_num}"]["mean"]).float().cuda()
        std = torch.tensor(norm_vals[f"Fold_{fold_num}"]["std"]).float().cuda()
        mean = mean.reshape(1, -1, 1, 1)
        std = std.reshape(1, -1, 1, 1)
        x_norm = (data - mean) / std
        
        T, C, H, W = x_norm.shape[1:]
        batch_positions = torch.linspace(0, 1, steps=T).unsqueeze(0).cuda()
        
        with torch.no_grad():
            output = model(x_norm, batch_positions=batch_positions)
            
            if return_raw_output:
                return output, (H, W)
            
        break
    
    return None, None

def load_model_fold(fold_num):
    """Charge un modèle pour un fold spécifique"""
    model = model_utils.get_model(config, mode="panoptic")
    model.to(device)
    
    checkpoint_path = os.path.join("../UTAE_PAPs", f"Fold_{fold_num}", "model.pth.tar")
    assert os.path.exists(checkpoint_path), f"Fichier introuvable : {checkpoint_path}"
    
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint["state_dict"])
    model.eval()
    return model.cuda()
    

In [58]:
def extract_instances_properties_final(data, tile_id, pixel_size=10, min_area_pixels=25, confidence_threshold=0.3):
    """Version modifiée utilisant pano_instance au lieu d'instance_masks"""
    
    instances_data = []
    
    # Convertir les tensors en numpy si nécessaire
    def to_numpy(x):
        if torch.is_tensor(x):
            return x.cpu().numpy()
        return x
    
    # Récupérer pano_instance au lieu d'instance_masks
    pano_instance = to_numpy(data['pano_instance'])  # (1, 128, 128) 
    semantic_logits = to_numpy(data['semantic'])      # (N, 20)
    confidence_scores = to_numpy(data['confidence'])  # (N, 1)
    instance_boxes = to_numpy(data['instance_boxes']) # (N, 4)
    
    # Traiter pano_instance
    if len(pano_instance.shape) == 3 and pano_instance.shape[0] == 1:
        pano_instance = pano_instance[0]  # (128, 128)
    
    # Aplatir confidence si nécessaire
    if len(confidence_scores.shape) > 1:
        confidence_scores = confidence_scores.flatten()
    
    print(f"📊 Composants extraits:")
    print(f"  Pano instance: {pano_instance.shape}")
    print(f"  Semantic logits: {semantic_logits.shape}")
    print(f"  Confidence scores: {confidence_scores.shape}")
    print(f"  Instance boxes: {instance_boxes.shape}")
    
    # Trouver les instances dans pano_instance
    unique_instances = np.unique(pano_instance)
    unique_instances = unique_instances[unique_instances > 0]  # Exclure background
    
    print(f"🔢 Instances trouvées dans pano_instance: {len(unique_instances)}")
    print(f"🔢 Prédictions sémantiques: {semantic_logits.shape[0]}")
    
    # Calculer les classes prédites
    predicted_classes = np.argmax(semantic_logits, axis=1)
    semantic_probs = np.exp(semantic_logits) / np.sum(np.exp(semantic_logits), axis=1, keepdims=True)
    class_probabilities = np.max(semantic_probs, axis=1)
    
    print(f"🏷️  Classes prédites: {np.unique(predicted_classes)}")
    print(f"📈 Confiances: min={confidence_scores.min():.3f}, max={confidence_scores.max():.3f}")
    
    # Traiter chaque instance trouvée dans pano_instance
    for i, instance_id in enumerate(unique_instances):
        try:
            # Créer le masque binaire pour cette instance
            instance_mask = (pano_instance == instance_id).astype(np.uint8)
            area_pixels = np.sum(instance_mask)
            
            # Filtrer les instances trop petites
            if area_pixels < min_area_pixels:
                continue
            
            print(f"  Instance {instance_id}: {area_pixels} pixels")
            
            # Associer avec les prédictions sémantiques
            # Stratégie: utiliser l'index de l'instance (i) pour associer aux logits
            if i < len(predicted_classes):
                class_id = predicted_classes[i]
                confidence = confidence_scores[i] if i < len(confidence_scores) else 1.0
                class_prob = class_probabilities[i] if i < len(class_probabilities) else 1.0
                bbox = instance_boxes[i] if i < len(instance_boxes) else [0, 0, 128, 128]
            else:
                # Fallback: essayer d'associer par ID si possible
                idx = min(int(instance_id) - 1, len(predicted_classes) - 1)
                idx = max(0, idx)
                class_id = predicted_classes[idx]
                confidence = confidence_scores[idx] if idx < len(confidence_scores) else 1.0
                class_prob = class_probabilities[idx] if idx < len(class_probabilities) else 1.0
                bbox = instance_boxes[idx] if idx < len(instance_boxes) else [0, 0, 128, 128]
            
            # Filtrer par confiance
            if confidence < confidence_threshold:
                print(f"    ❌ Confiance trop faible: {confidence:.3f}")
                continue
            
            # Calculer les propriétés géométriques (même fonction que vous utilisez déjà)
            props = calculate_instance_properties_corrected(
                instance_mask, tile_id, int(instance_id), class_id, confidence, class_prob, bbox, pixel_size
            )
            
            if props:
                instances_data.append(props)
                print(f"    ✅ Instance {instance_id} extraite: classe {class_id}, {props['area_ha']:.4f} ha")
            
        except Exception as e:
            print(f"    ⚠️ Erreur pour instance {instance_id}: {e}")
            continue
    
    print(f"✅ {len(instances_data)} instances extraites avec succès")
    return instances_data

def calculate_instance_properties_corrected(mask, tile_id, instance_id, class_id, confidence, class_prob, bbox, pixel_size=10):
    """Version corrigée du calcul des propriétés"""
    
    # S'assurer que le masque est en numpy
    if torch.is_tensor(mask):
        mask = mask.cpu().numpy()
    
    # Convertir en masque binaire
    if mask.dtype != np.uint8:
        # Si c'est un masque de probabilité, seuiller
        if mask.max() <= 1.0 and mask.min() >= 0.0:
            binary_mask = (mask > 0.5).astype(np.uint8)
        else:
            # Si c'est déjà des valeurs entières
            binary_mask = (mask > 0).astype(np.uint8)
    else:
        binary_mask = (mask > 0).astype(np.uint8)
    
    print(f"    Masque: {mask.shape}, dtype: {mask.dtype}, min: {mask.min()}, max: {mask.max()}")
    print(f"    Masque binaire: sum: {binary_mask.sum()}")
    
    # Vérifier que le masque n'est pas vide
    if np.sum(binary_mask) == 0:
        print(f"    ❌ Masque vide")
        return None
    
    # Aire en pixels
    area_pixels = np.sum(binary_mask)
    area_m2 = area_pixels * (pixel_size ** 2)
    area_ha = area_m2 / 10000
    
    # Filtrer les instances trop petites
    if area_pixels < 10:  # Minimum 10 pixels
        print(f"    ❌ Instance trop petite: {area_pixels} pixels")
        return None
    
    # Trouver les contours
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        print(f"    ❌ Aucun contour trouvé")
        return None
    
    # Prendre le plus grand contour
    main_contour = max(contours, key=cv2.contourArea)
    contour_area = cv2.contourArea(main_contour)
    
    if contour_area < 5:
        print(f"    ❌ Contour trop petit: {contour_area}")
        return None
    
    # Périmètre
    perimeter_pixels = cv2.arcLength(main_contour, True)
    perimeter_m = perimeter_pixels * pixel_size
    
    # Centre de masse (centroïde)
    M = cv2.moments(main_contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
    else:
        # Fallback: centre de masse du masque
        y_coords, x_coords = np.where(binary_mask > 0)
        cx = int(np.mean(x_coords)) if len(x_coords) > 0 else 0
        cy = int(np.mean(y_coords)) if len(y_coords) > 0 else 0
    
    # Boîte englobante du contour
    x, y, w, h = cv2.boundingRect(main_contour)
    
    # Boîte englobante prédite par le modèle
    bbox_x1, bbox_y1, bbox_x2, bbox_y2 = bbox
    bbox_w_pred = bbox_x2 - bbox_x1
    bbox_h_pred = bbox_y2 - bbox_y1
    
    # Ellipse ajustée
    if len(main_contour) >= 5:
        try:
            ellipse = cv2.fitEllipse(main_contour)
            ellipse_center = ellipse[0]
            ellipse_axes = ellipse[1]
            ellipse_angle = ellipse[2]
        except:
            ellipse_center = (cx, cy)
            ellipse_axes = (w, h)
            ellipse_angle = 0
    else:
        ellipse_center = (cx, cy)
        ellipse_axes = (w, h)
        ellipse_angle = 0
    
    # Indicateurs de forme
    compactness = (4 * np.pi * contour_area) / (perimeter_pixels ** 2) if perimeter_pixels > 0 else 0
    aspect_ratio = max(w, h) / min(w, h) if min(w, h) > 0 else 1
    
    # Convexité
    hull = cv2.convexHull(main_contour)
    hull_area = cv2.contourArea(hull)
    convexity = contour_area / hull_area if hull_area > 0 else 0
    
    # Solidité
    solidity = contour_area / hull_area if hull_area > 0 else 0
    
    # Extent
    extent = contour_area / (w * h) if (w * h) > 0 else 0
    
    # Coordonnées du polygone
    contour_points = main_contour.reshape(-1, 2).tolist()
    
    # Diamètre équivalent
    equivalent_diameter = np.sqrt(4 * contour_area / np.pi)
    
    # Retourner avec types JSON-safe
    return {
        'tile_id': str(tile_id),
        'instance_id': f"{tile_id}_{instance_id}",
        'class_id': int(class_id),
        'confidence': float(confidence),
        'class_probability': float(class_prob),
        
        # Centre
        'centroid_x': int(cx),
        'centroid_y': int(cy),
        'centroid_x_m': float(cx * pixel_size),
        'centroid_y_m': float(cy * pixel_size),
        
        # Surface
        'area_pixels': float(area_pixels),
        'area_m2': float(area_m2),
        'area_ha': float(area_ha),
        'contour_area_pixels': float(contour_area),
        
        # Périmètre
        'perimeter_pixels': float(perimeter_pixels),
        'perimeter_m': float(perimeter_m),
        
        # Boîte englobante (contour réel)
        'bbox_x': int(x), 'bbox_y': int(y), 'bbox_w': int(w), 'bbox_h': int(h),
        'bbox_width_m': float(w * pixel_size),
        'bbox_height_m': float(h * pixel_size),
        
        # Boîte englobante prédite
        'predicted_bbox_x1': float(bbox_x1),
        'predicted_bbox_y1': float(bbox_y1),
        'predicted_bbox_x2': float(bbox_x2),
        'predicted_bbox_y2': float(bbox_y2),
        'predicted_bbox_width_m': float(bbox_w_pred * pixel_size),
        'predicted_bbox_height_m': float(bbox_h_pred * pixel_size),
        
        # Ellipse
        'ellipse_center_x': float(ellipse_center[0]),
        'ellipse_center_y': float(ellipse_center[1]),
        'ellipse_major_axis_m': float(max(ellipse_axes) * pixel_size),
        'ellipse_minor_axis_m': float(min(ellipse_axes) * pixel_size),
        'ellipse_angle': float(ellipse_angle),
        
        # Indicateurs de forme
        'compactness': float(compactness),
        'aspect_ratio': float(aspect_ratio),
        'convexity': float(convexity),
        'solidity': float(solidity),
        'extent': float(extent),
        'equivalent_diameter_m': float(equivalent_diameter * pixel_size),
        
        # Géométrie
        'polygon_coords': [[int(pt[0]), int(pt[1])] for pt in contour_points],
        'n_vertices': int(len(contour_points))
    }

In [ ]:
def run_evaluation_on_training_tile(tile_id, pastis_root, confidence_threshold=0.3):
    """Évalue tous les folds sur une tuile d'entraînement"""
    
    print(f"🎯 ÉVALUATION SUR DONNÉES D'ENTRAÎNEMENT - Tuile {tile_id}")
    print("=" * 60)
    
    # 1. Charger les annotations ground truth
    annotations = load_pastis_annotations_for_evaluation(tile_id, pastis_root)
    if not annotations:
        print("❌ Impossible de charger les annotations")
        return None
    
    gt_instances = extract_ground_truth_instances(annotations, tile_id)
    if not gt_instances:
        print("❌ Aucune instance GT extraite")
        return None
    
    print(f"🎯 Ground Truth: {len(gt_instances)} parcelles")
    
    # 2. Inférence sur tous les folds
    fold_results = {}
    
    for fold_num in range(1, 6):
        print(f"\n🔄 Évaluation Fold {fold_num}")
        print("-" * 30)
        
        try:
            # Faire l'inférence
            output, image_shape = load_model_and_predict(tile_id, fold_num, return_raw_output=True)
            
            if output is None:
                print(f"  ❌ Échec de l'inférence")
                continue
            
            # Extraire les instances prédites
            pred_instances = extract_instances_properties_corrected(output, tile_id)
            
            # Filtrer par confiance
            pred_instances_filtered = [
                inst for inst in pred_instances 
                if inst['confidence'] >= confidence_threshold
            ]
            
            print(f"  📊 Prédictions: {len(pred_instances)} total, {len(pred_instances_filtered)} après filtrage")
            
            # Évaluer contre GT
            if pred_instances_filtered:
                metrics = evaluate_predictions_vs_ground_truth(
                    pred_instances_filtered, gt_instances, tile_id, fold_num
                )
                fold_results[fold_num] = {
                    'predictions': pred_instances_filtered,
                    'metrics': metrics,
                    'n_predictions': len(pred_instances_filtered)
                }
            else:
                fold_results[fold_num] = {
                    'predictions': [],
                    'metrics': None,
                    'n_predictions': 0
                }
                
        except Exception as e:
            print(f"  ❌ Erreur fold {fold_num}: {e}")
            fold_results[fold_num] = {'error': str(e)}
    
    # 3. Résumé comparatif
    comparison_results = {
        'tile_id': tile_id,
        'ground_truth': gt_instances,
        'fold_results': fold_results,
        'n_gt_instances': len(gt_instances)
    }
    
    print_fold_comparison(fold_results, tile_id)
    
    return comparison_results

def evaluate_predictions_vs_ground_truth(pred_instances, gt_instances, tile_id, fold_num):
    """Évalue les prédictions contre le ground truth"""
    
    print(f"  📊 Évaluation Fold {fold_num}:")
    print(f"    Prédictions: {len(pred_instances)}")
    print(f"    Ground Truth: {len(gt_instances)}")
    
    if len(pred_instances) == 0 or len(gt_instances) == 0:
        return {
            'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0,
            'mean_iou': 0.0, 'classification_accuracy': 0.0,
            'n_matches': 0, 'n_predictions': len(pred_instances), 'n_ground_truth': len(gt_instances)
        }
    
    # 1. Calculer les IoU entre toutes les paires
    iou_matrix = compute_iou_matrix_instances(pred_instances, gt_instances)
    
    # 2. Assignment optimal (Hungarian algorithm)
    cost_matrix = 1 - iou_matrix
    pred_indices, gt_indices = linear_sum_assignment(cost_matrix)
    
    # 3. Filtrer les assignments avec IoU suffisant
    iou_threshold = 0.3
    valid_matches = []
    
    for p_idx, g_idx in zip(pred_indices, gt_indices):
        iou = iou_matrix[p_idx, g_idx]
        if iou >= iou_threshold:
            pred_class = pred_instances[p_idx]['class_id']
            gt_class = gt_instances[g_idx]['class_id']
            
            match = {
                'pred_idx': p_idx,
                'gt_idx': g_idx,
                'iou': iou,
                'pred_class': pred_class,
                'gt_class': gt_class,
                'class_correct': (pred_class == gt_class),
                'confidence': pred_instances[p_idx]['confidence']
            }
            valid_matches.append(match)
    
    # 4. Calculer les métriques
    n_matches = len(valid_matches)
    precision = n_matches / len(pred_instances) if len(pred_instances) > 0 else 0
    recall = n_matches / len(gt_instances) if len(gt_instances) > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    # IoU moyen des matches
    mean_iou = np.mean([m['iou'] for m in valid_matches]) if valid_matches else 0.0
    
    # Accuracy de classification sur les matches
    correct_classifications = sum(1 for m in valid_matches if m['class_correct'])
    classification_accuracy = correct_classifications / n_matches if n_matches > 0 else 0.0
    
    metrics = {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'mean_iou': mean_iou,
        'classification_accuracy': classification_accuracy,
        'n_matches': n_matches,
        'n_predictions': len(pred_instances),
        'n_ground_truth': len(gt_instances),
        'matches': valid_matches
    }
    
    print(f"    📈 Precision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1_score:.3f}")
    print(f"    📐 IoU moyen: {mean_iou:.3f}, Acc. classification: {classification_accuracy:.3f}")
    
    return metrics

def compute_iou_matrix_instances(pred_instances, gt_instances):
    """Calcule la matrice IoU entre prédictions et ground truth"""
    
    n_pred = len(pred_instances)
    n_gt = len(gt_instances)
    iou_matrix = np.zeros((n_pred, n_gt))
    
    for i, pred in enumerate(pred_instances):
        pred_coords = np.array(pred['polygon_coords'])
        
        for j, gt in enumerate(gt_instances):
            gt_coords = np.array(gt['polygon_coords'])
            
            # Calculer IoU avec Shapely
            iou = compute_polygon_iou_shapely(pred_coords, gt_coords)
            iou_matrix[i, j] = iou
    
    return iou_matrix

def compute_polygon_iou_shapely(poly1_coords, poly2_coords):
    """Calcule l'IoU entre deux polygones avec Shapely"""
    
    try:
        from shapely.geometry import Polygon
        
        if len(poly1_coords) < 3 or len(poly2_coords) < 3:
            return 0.0
        
        p1 = Polygon(poly1_coords)
        p2 = Polygon(poly2_coords)
        
        # Nettoyer les polygones si nécessaire
        if not p1.is_valid:
            p1 = p1.buffer(0)
        if not p2.is_valid:
            p2 = p2.buffer(0)
        
        if p1.is_valid and p2.is_valid and not p1.is_empty and not p2.is_empty:
            intersection = p1.intersection(p2)
            union = p1.union(p2)
            
            if union.area > 0:
                return intersection.area / union.area
        
        return 0.0
        
    except Exception:
        return 0.0

In [53]:
def print_fold_comparison(fold_results, tile_id):
    """Affiche la comparaison entre les folds"""
    
    print(f"\n📊 COMPARAISON DES FOLDS - Tuile {tile_id}")
    print("=" * 60)
    
    # Tableau comparatif
    print("Fold | Pred | F1-Score | Precision | Recall | IoU moy | Class Acc | Matches")
    print("-" * 75)
    
    fold_scores = {}
    
    for fold_num in range(1, 6):
        if fold_num in fold_results and 'metrics' in fold_results[fold_num]:
            if fold_results[fold_num]['metrics']:
                m = fold_results[fold_num]['metrics']
                n_pred = m['n_predictions']
                f1 = m['f1_score']
                precision = m['precision']
                recall = m['recall']
                iou = m['mean_iou']
                acc = m['classification_accuracy']
                matches = m['n_matches']
                
                print(f" {fold_num}   | {n_pred:4d} | {f1:8.3f} | {precision:9.3f} | {recall:6.3f} | {iou:7.3f} | {acc:9.3f} | {matches:7d}")
                
                # Score global combiné
                global_score = (f1 + iou + acc) / 3
                fold_scores[fold_num] = global_score
            else:
                print(f" {fold_num}   |    0 |    0.000 |     0.000 |  0.000 |   0.000 |     0.000 |       0")
        else:
            print(f" {fold_num}   | ERR  |      ERR |       ERR |    ERR |     ERR |       ERR |     ERR")
    
    # Meilleur fold
    if fold_scores:
        best_fold = max(fold_scores, key=fold_scores.get)
        print(f"\n🏆 MEILLEUR FOLD: Fold {best_fold} (score global: {fold_scores[best_fold]:.3f})")
        
        return best_fold, fold_scores
    
    return None, {}

def run_evaluation_on_multiple_tiles(tile_ids, pastis_root):
    """Évalue sur plusieurs tuiles d'entraînement"""
    
    print(f"🎯 ÉVALUATION SUR {len(tile_ids)} TUILES D'ENTRAÎNEMENT")
    print("=" * 60)
    
    all_results = {}
    fold_performance_summary = {i: [] for i in range(1, 6)}
    
    for tile_id in tile_ids:
        print(f"\n{'='*80}")
        results = run_evaluation_on_training_tile(tile_id, pastis_root)
        
        if results:
            all_results[tile_id] = results
            
            # Collecter les scores de performance
            for fold_num in range(1, 6):
                if fold_num in results['fold_results'] and 'metrics' in results['fold_results'][fold_num]:
                    metrics = results['fold_results'][fold_num]['metrics']
                    if metrics:
                        global_score = (metrics['f1_score'] + metrics['mean_iou'] + metrics['classification_accuracy']) / 3
                        fold_performance_summary[fold_num].append(global_score)
                    else:
                        fold_performance_summary[fold_num].append(0.0)
    
    # Analyse globale
    print(f"\n🏆 ANALYSE GLOBALE SUR {len(tile_ids)} TUILES")
    print("=" * 60)
    
    print("Fold | Moy Score | Médiane | Std Dev | Min | Max | Nb Succès")
    print("-" * 65)
    
    fold_rankings = {}
    
    for fold_num in range(1, 6):
        scores = fold_performance_summary[fold_num]
        if scores:
            mean_score = np.mean(scores)
            median_score = np.median(scores)
            std_score = np.std(scores)
            min_score = np.min(scores)
            max_score = np.max(scores)
            success_rate = sum(1 for s in scores if s > 0.1) / len(scores)  # Seuil arbitraire
            
            print(f" {fold_num}   | {mean_score:9.3f} | {median_score:7.3f} | {std_score:7.3f} | {min_score:.3f} | {max_score:.3f} | {success_rate:9.1%}")
            fold_rankings[fold_num] = mean_score
        else:
            print(f" {fold_num}   |     0.000 |   0.000 |   0.000 | 0.000 | 0.000 |     0.0%")
    
    # Recommandation finale
    if fold_rankings:
        best_overall_fold = max(fold_rankings, key=fold_rankings.get)
        print(f"\n🎉 RECOMMANDATION: Utilisez le Fold {best_overall_fold}")
        print(f"   Score moyen: {fold_rankings[best_overall_fold]:.3f}")
        print(f"   Écart avec le 2e meilleur: {fold_rankings[best_overall_fold] - sorted(fold_rankings.values())[-2]:.3f}")
    
    # Sauvegarder les résultats
    with open('fold_evaluation_results.json', 'w') as f:
        json.dump(all_results, f, indent=2, default=str)
    
    return all_results, fold_rankings

# Exemple d'utilisation
if __name__ == "__main__":
    # Liste des tuiles d'entraînement à évaluer
    training_tile_ids = ["10000", "10001"]  # Adaptez selon vos données
    pastis_root = "s3://antoinelesauvage/zenodo/PASTIS_unz/PASTIS"
    
    # Lancer l'évaluation
    results, rankings = run_evaluation_on_multiple_tiles(training_tile_ids, pastis_root)

🎯 ÉVALUATION SUR 2 TUILES D'ENTRAÎNEMENT

🎯 ÉVALUATION SUR DONNÉES D'ENTRAÎNEMENT - Tuile 10000
📂 Chargement annotations PASTIS pour évaluation - Tuile 10000
  ✅ parcels: shape (128, 128), dtype int32
  ✅ targets: shape (3, 128, 128), dtype uint8
📊 Extraction GT - Tuile 10000
  Parcelles: (128, 128)
  Targets: (3, 128, 128)
  Traitement de 116 parcelles...
  ✅ 107 parcelles GT extraites
🎯 Ground Truth: 107 parcelles

🔄 Évaluation Fold 1
------------------------------
📊 Composants extraits:
  Instance masks: 195 masques (liste)
  Semantic logits: (195, 20)
  Confidence scores: (195,)
  Instance boxes: (195, 4)
🏷️  Classes prédites: [ 1  3  4  5  6 11 12 14 16]
📈 Confiances: min=0.010, max=0.968, mean=0.318
🎯 Probabilités de classe: min=0.191, max=0.975
🔢 Traitement de 195 instances...
  Instance 0: classe=1, conf=0.016, shape=(1, 1)
    ❌ Confiance trop faible: 0.016
  Instance 1: classe=16, conf=0.010, shape=(1, 1)
    ❌ Confiance trop faible: 0.010
  Instance 2: classe=1, conf=0.027, 

In [ ]:
# Script principal pour l'évaluation des folds
def main_processing_with_pano():
    """Script principal utilisant pano_instance"""
    
    print("🚀 TRAITEMENT AVEC PANO_INSTANCE")
    print("=" * 60)
    
    # 1. Test sur un fichier
    print("Phase 1: Test sur un fichier")
    test_result = test_single_file_with_pano("preds/0_best_fold_5.npy")
    
    if test_result:
        print("✅ Test réussi! Lancement du traitement complet...")
        
        # 2. Traiter tous les fichiers
        df_all, summary = process_all_prediction_files_with_pano()
        
        if df_all is not None:
            # 3. Analyser les résultats (réutiliser votre fonction d'analyse)
            analyze_final_results(df_all)
            
            print(f"\n🎉 TRAITEMENT TERMINÉ!")
            print(f"📊 {len(df_all)} instances extraites avec pano_instance")
            print(f"💾 Données sauvegardées dans all_instances_pano.csv")
            
            return df_all, summary
        else:
            print("❌ Aucune instance extraite")
            return None, None
    else:
        print("❌ Échec du test initial")
        return None, None

# Lancer le traitement complet
df_final, summary_final = main_processing_with_pano()

🚀 ÉVALUATION DES FOLDS SUR DONNÉES D'ENTRAÎNEMENT
Phase 1: Test sur une tuile
🎯 ÉVALUATION SUR DONNÉES D'ENTRAÎNEMENT - Tuile 10000
📂 Chargement annotations PASTIS pour évaluation - Tuile 10000
  ✅ parcels: shape (128, 128), dtype int32
  ✅ targets: shape (3, 128, 128), dtype uint8
📊 Extraction GT - Tuile 10000
  Parcelles: (128, 128)
  Targets: (3, 128, 128)
  Traitement de 116 parcelles...
  ✅ 107 parcelles GT extraites
🎯 Ground Truth: 107 parcelles

🔄 Évaluation Fold 1
------------------------------
📊 Composants extraits:
  Instance masks: 195 masques (liste)
  Semantic logits: (195, 20)
  Confidence scores: (195,)
  Instance boxes: (195, 4)
🏷️  Classes prédites: [ 1  3  4  5  6 11 12 14 16]
📈 Confiances: min=0.010, max=0.968, mean=0.318
🎯 Probabilités de classe: min=0.191, max=0.975
🔢 Traitement de 195 instances...
  Instance 0: classe=1, conf=0.016, shape=(1, 1)
    ❌ Confiance trop faible: 0.016
  Instance 1: classe=16, conf=0.010, shape=(1, 1)
    ❌ Confiance trop faible: 0.010
